##  Step 1: Define the Elliptic Curve Parameters
We'll use a simple elliptic curve defined over a finite field:
- Curve Equation = y^2 = x^3 + ax + b 
- Parameter a,b and a prime p


In [2]:
import os

# Elliptic Curve parameters
a = 2
b = 3
p = 97  # A small prime number for simplicity


# Check if a point is on the curve
def is_point_on_curve(x, y):
    return (y ** 2) % p == (x ** 3 + a * x + b) % p

In [3]:
# Point addition
def point_add(P, Q):
    if P == (0, 0):  # Identity point
        return Q
    if Q == (0, 0):  # Identity point
        return P

    x1, y1 = P
    x2, y2 = Q

    if x1 == x2 and y1 == -y2 % p:
        return (0, 0)  # Point at infinity

    if P != Q:
        m = (y2 - y1) * pow(x2 - x1, p - 2, p) % p  # Modular inverse
    else:
        m = (3 * x1 ** 2 + a) * pow(2 * y1, p - 2, p) % p  # Derivative at P

    x3 = (m ** 2 - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p

    return (x3, y3)


# Scalar multiplication
def scalar_multiply(k, P):
    R = (0, 0)  # Identity point
    while k:
        if k & 1:  # If k is odd
            R = point_add(R, P)
        P = point_add(P, P)  # Double the point
        k >>= 1
    return R



## Step 3: Key Generation

Generate a public/private key pair:

In [4]:


def generate_keypair():
    private_key = int.from_bytes(os.urandom(32), byteorder='big') % (p - 1)  # Private key
    G = (3, 6)  # Example generator point on the curve
    public_key = scalar_multiply(private_key, G)  # Public key
    return private_key, public_key


## Step 4: Encrypt and Decrypt a Message

We'll implement a simple message encryption scheme using the shared secret generated from the public key

In [5]:

def encrypt(message, public_key):
    G = (3, 6)  # Generator point
    k = int.from_bytes(os.urandom(32), byteorder='big') % (p - 1)  # Random scalar

    C1 = scalar_multiply(k, G)  # C1 = kG
    C2 = scalar_multiply(k, public_key)  # C2 = kP (shared secret)

    # Encrypt the message (simple XOR for demonstration)
    encrypted_message = bytes((m ^ C2[0]) % 256 for m in message)
    return C1, encrypted_message


def decrypt(ciphertext, private_key):
    C1, encrypted_message = ciphertext
    C2 = scalar_multiply(private_key, C1)  # C2 = dC1 (shared secret)

    # Decrypt the message
    decrypted_message = bytes((em ^ C2[0]) % 256 for em in encrypted_message)
    return decrypted_message



## Step 5: Putting It All Together

Now, we can create a main function to demonstrate the whole process:

In [6]:

def main():
    # Key generation
    private_key, public_key = generate_keypair()
    print(f"Private Key: {private_key}")
    print(f"Public Key: {public_key}")

    # Message to encrypt
    message = b"Hello, this is a secret message!"

    # Encrypt the message
    ciphertext = encrypt(message, public_key)
    print(f"Ciphertext: {ciphertext}")

    # Decrypt the message
    decrypted_message = decrypt(ciphertext, private_key)
    print(f"Decrypted Message: {decrypted_message.decode()}")


if __name__ == "__main__":
    main()

Private Key: 42
Public Key: (80, 10)
Ciphertext: ((3, 6), b'\x185<<?|p$89#p9#p1p#53"5$p=5##175q')
Decrypted Message: Hello, this is a secret message!
